<a href="https://colab.research.google.com/github/Finardiprof/Python/blob/main/Fermi_LAT_data_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tutorial per l'esplorazione dei dati di navigazione di Fermi-LAT**

#### **Tutorial a cura di D. Serini**
###### **con il contributo di E. Bissaldi, F. de Palma, L. Di Venere, F. Gargano**
#### Contatti: [email](mailto:astroparticellebari@gmail.com)


Questo tutorial mostra come esplorare i dati di navigazione del satellite Fermi-LAT, indicati solitamente con il nome "*FT2*".

#### ISTRUZIONI

La piattaforma che stiamo utilizzando si chima Google Colab, che fornisce un *python-notebook*, cioè una interfaccia che ci consente di eseguire comandi nel linguaggio di programmazione [python](https://it.wikipedia.org/wiki/Python)  e visualizzare dati e grafici.

Per utilizzare il *notebook*: ogni blocco di comandi può essere eseguito cliccando sul tasto "play" situato in alto a sinistra della cella, oppure ponendo il cursore nella cella e cliccando Shift+Invio oppure Ctrl+Invio.
All'esecuzione del primo blocco comparirà una finestra di pop-up, nella quale bisogna cliccare sul tasto "Run anyway".

Il *notebook* può essere modificato, ma le modifiche non verranno salvate. Per salvare le proprie modifiche è necessario salvare una copia del *notebook* nel proprio account GoogleDrive, cliccando sul tasto "Copy to Drive", posizionato in alto sotto la barra dei Menù.

Importiamo le librerie necessarie alla lettura e rappresentazione dei dati

In [1]:
%matplotlib inline
import os,sys
import time, datetime
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord, search_around_sky
from astropy.visualization.wcsaxes import SphericalCircle

from matplotlib import patheffects
from matplotlib.colors import LogNorm
from matplotlib.dates import DateFormatter
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage,AnnotationBbox
from IPython.display import display, clear_output, Image
from IPython.utils.text import columnize
from ipywidgets import interactive, interact
import ipywidgets as widgets

!pip install healpy >& /dev/null
import healpy as hp
!pip install geopandas >& /dev/null
import geopandas

import warnings
warnings.simplefilter("ignore")

print("Tutte le librerie sono state correttamente importate")


Tutte le librerie sono state correttamente importate



---
# Dati
---

**Scarichiamo i dati da un server della NASA, potrebbe volerci qualche minuto, non preoccupatevi!**


In [2]:
## Download data
filename_ft2 = "FERMI_POINTING_FINAL_604_2019360_2020002_00.fits" ## Dec '19
filename_ft2 = "FERMI_POINTING_FINAL_044_2009092_2009099_00.fits" ## Apr '09

spacecraft="Fermi_GLAST.png"
allskymapHpx = "Allsky_map_HPX.fits"

!wget https://fermi.gsfc.nasa.gov/ssc/observations/timeline/ft2/files/FERMI_POINTING_FINAL_604_2019360_2020002_00.fits >& /dev/null
!wget https://fermi.gsfc.nasa.gov/ssc/observations/timeline/ft2/files/FERMI_POINTING_FINAL_044_2009092_2009099_00.fits >& /dev/null
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=16wQAir7NTEHBaNU2tRXkFdwrYQt7LG46' -O $spacecraft >& /dev/null
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Cyxr788w5N8xzXSbwvSP-EdydUCy4wBX' -O $allskymapHpx  >& /dev/null

print("Il contenuto della cartella è:")
!ls -lrt


Il contenuto della cartella è:
total 7376
-rw-r--r-- 1 root root  993600 May 19  2010 FERMI_POINTING_FINAL_044_2009092_2009099_00.fits
-rw-r--r-- 1 root root 1497600 Dec 20  2019 FERMI_POINTING_FINAL_604_2019360_2020002_00.fits
-rw-r--r-- 1 root root 1321920 Apr  6  2020 Allsky_map_HPX.fits
-rw-r--r-- 1 root root 3730306 Apr  9  2020 Fermi_GLAST.png
drwxr-xr-x 1 root root    4096 Oct 16 13:23 sample_data


---
Apriamo il file *FT2* e carichiamo i dati. Si tratta di una tabella che contiene le coordinate del satellite al variare del tempo.
Ecco alcune delle informazioni contenute nella tabella:

*   Tempo: definito come tempo di inizio (**START**) e tempo di fine (**STOP**)
*   Posizione del satellite sulla Terra: [latitudine](https://it.wikipedia.org/wiki/Latitudine) (**LAT_GEO**) e [longitudine](https://it.wikipedia.org/wiki/Longitudine) (**LON_GEO**)
*   Direzione del cielo al quale il satellite punta, identificata tramite un sistema di coordinate, dette coordinate celesti: [Ascensione retta](https://it.wikipedia.org/wiki/Ascensione_retta) (**RA_SCZ**) e [Declinazione](https://it.wikipedia.org/wiki/Declinazione_(astronomia)) (**DEC_SCZ**)
*   Posizione del Sole nel cielo, identificata sempre con coordinate celesti: **RA_SUN** e **DEC_SUN**

Abbiamo scaricato due file corrispondenti a due periodi di osservazione diversi. Possiamo commentare la prima riga (aggiungendo un '#' all'inizio) e scommentare la seconda (eliminando il '#') per vedere come cambiano i grafici successivi. Una volta fatta questa modifica è necessario eseguire nuovamente tutti i blocchi successivi per notare le differenze.




In [ ]:
# Apriamo il file
filename_ft2 = "FERMI_POINTING_FINAL_604_2019360_2020002_00.fits" ## Dec '19
#filename_ft2 = "FERMI_POINTING_FINAL_044_2009092_2009099_00.fits" ## Apr '09

hdu_ft2=pyfits.open(filename_ft2)
# Stampiamo le informazioni generali del file
print("Il contenuto del file è:")
hdu_ft2.info()
# Carichiamo i dati nella tabella chiamata 'SC_DATA'
tab_ft2 = Table(hdu_ft2['SC_DATA'].data)
print()
# Stampiamo i nomi delle colonne contenute nella tabella
print("Le colonne nella tabella sono:")
print(columnize(tab_ft2.colnames))

# Selezioniamo alcune colonne e visualizziamo le prime righe della tabella
print("Dati:")
max_entries = min(50,len(tab_ft2))
tab_ft2['START','STOP','LAT_GEO', 'LON_GEO', 'RAD_GEO','RA_SUN', 'DEC_SUN', 'IN_SAA'][:max_entries].show_in_notebook(display_length=50)
# Prova a cambiare i nomi delle colonne inseriti nella riga precedente, scegliendoli tra i nomi delle colonne visualizzati qui in basso.

Il contenuto del file è:
Filename: FERMI_POINTING_FINAL_604_2019360_2020002_00.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      27   ()      
  1  SC_DATA       1 BinTableHDU    143   10081R x 29C   [D, D, 3E, E, E, D, E, E, E, E, E, L, E, E, E, E, E, E, E, J, B, B, D, D, D, D, D, E, E]   

Le colonne nella tabella sono:
START        RAD_GEO     GEOMAG_LAT  DEC_SCX     LAT_CONFIG  QSJ_3  
STOP         RA_ZENITH   IN_SAA      RA_NPOLE    DATA_QUAL   QSJ_4  
SC_POSITION  DEC_ZENITH  RA_SCZ      DEC_NPOLE   LIVETIME    RA_SUN 
LAT_GEO      B_MCILWAIN  DEC_SCZ     ROCK_ANGLE  QSJ_1       DEC_SUN
LON_GEO      L_MCILWAIN  RA_SCX      LAT_MODE    QSJ_2     

Dati:


idx,START,STOP,LAT_GEO,LON_GEO,RAD_GEO,RA_SUN,DEC_SUN,IN_SAA
0,599011140.0,599011200.0,3.604,121.664,536258.0539151104,274.25467,-23.379433,False
1,599011200.0,599011260.0,5.224,124.839,535718.4440664337,274.25546,-23.379414,False
2,599011260.0,599011320.0,6.825,128.033,535181.8969221233,274.25623,-23.379393,False
3,599011320.0,599011380.0,8.402,131.249,534649.457072667,274.257,-23.379372,False
4,599011380.0,599011440.0,9.948,134.494,534121.7953551566,274.25775,-23.37935,False
5,599011440.0,599011500.0,11.458,137.772,533599.2800259524,274.25854,-23.37933,False
6,599011500.0,599011560.0,12.925,141.088,533081.9854004608,274.2593,-23.379309,False
7,599011560.0,599011620.0,14.343,144.445,532569.720634104,274.26007,-23.379288,False
8,599011620.0,599011680.0,15.706,147.848,532062.0755687836,274.26083,-23.379267,False
9,599011680.0,599011740.0,17.007,151.299,531558.5081573105,274.26163,-23.379246,False


---

# Il tempo per il satellite Fermi

Gli istanti di tempo sono codificati in MET = Mission Elapsed Time, ovvero il tempo trascorso dall'inizio della missione.
Si è fissata convenzionalmente l'inizio della missione al primo gennaio del 2001, molto prima della data di lancio del satellite, avvenuta nel giugno 2008.
In effetti la progettazione e la costruzione di Fermi sono iniziati ben prima del lancio!



In [ ]:
## Conversione dei tempi
met0 = datetime.datetime(2001,1,1,0,0,0)
def met2date(met_seconds):
  return met0 + met_seconds*datetime.timedelta(seconds=1)

## Leggiamo i tempi dal file FT2
start = tab_ft2['START']
stop = tab_ft2['STOP']
start_date = met2date(start)
stop_date = met2date(stop)
print("L'inizio del file FT2 è:",start_date[0])
print("La  fine del file FT2 è:",stop_date[-1])

L'inizio del file FT2 è: 2019-12-25 23:59:00
La  fine del file FT2 è: 2020-01-02 00:00:00



---
# Traiettoria del satellite Fermi

Il file FT2 contiene la posizione del satellite al variare del tempo. Tale informazione è contenuto nelle colonne **LON_GEO** e **LAT_GEO** che rappresentano la longitudine e la latitudine del satellite. Leggiamo tali dati e rappresentiamoli.

Potremo notare che il satellite segue una orbita equatoriale e che la sua latitudine è sempre compresa tra 25°S e 25°N.


In [ ]:
## Traiettoria del satellite
# Mappa interattiva
# Esegui questo blocco e poi prova a modificare la variabile tempo usando la barra interattiva sul grafico
print("Cambia la variabile tempo per visualizzare la traiettoria del satellite")
print("Alcuni valori da provare: 50, 100, 200, 500, 1000, 1500 espressi in minuti")
lat_geo = tab_ft2['LAT_GEO']
lon_geo = tab_ft2['LON_GEO']
max_entries=len(tab_ft2)-1
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
arr_img = plt.imread(spacecraft, format='png')

play = widgets.Play(value=50,min=0,max=100,step=1,interval=500,description="Press play",disabled=False)
intslider = widgets.IntSlider(min=0, max=max_entries, step=1, value=0, description="Tempo (min)",continuous_update=False)
def plot_world(t):
  fig,ax = plt.subplots(figsize=(14,12));
  imagebox = OffsetImage(arr_img, zoom=0.04)
  imagebox.image.axes = ax
  world.plot(ax=ax)
  xy = [lon_geo[t],lat_geo[t]]
  ab = AnnotationBbox(imagebox, xy,xybox=(4., 2.),xycoords='data',boxcoords="offset points",pad=0.5, frameon=False)
  ax.add_artist(ab)

  xx = lon_geo[:t+1]
  yy = lat_geo[:t+1]
  idx = np.where(np.abs(np.diff(xx))>180)[0].tolist()
  idx = [0]+idx+[len(xx)]
  tmp_xx = xx
  tmp_yy = yy
  for i in range(len(idx)-1):
    tmp_xx=xx[idx[i]+1:idx[i+1]]
    tmp_yy=yy[idx[i]+1:idx[i+1]]
    ax.plot(tmp_xx,tmp_yy,'k--')
  plt.show()

w = interactive(plot_world, t=intslider)
output = w.children[-1]
output.layout.height = '410px'
display(w)


Cambia la variabile tempo per visualizzare la traiettoria del satellite
Alcuni valori da provare: 50, 100, 200, 500, 1000, 1500 espressi in minuti


interactive(children=(IntSlider(value=0, continuous_update=False, description='Tempo (min)', max=10080), Outpu…

# Altitudine di Fermi
L'altitudine del satellite è circa costante, intorno al valore di 530 km.

In [ ]:
time = met2date((tab_ft2['START']+tab_ft2['STOP'])/2)
alt = tab_ft2['RAD_GEO']/1000. ## km
alt_media = alt.mean()
print("L'altitudine media è {0:.2f} km".format(alt_media))

play = widgets.Play(value=0,min=0,max=max_entries,step=1,interval=100,description="Press play",disabled=False)
intslider = widgets.IntSlider(min=0, max=max_entries, step=1, value=0,description="Tempo (min)", continuous_update=False)

def plot_altitude(i):
  fig,ax = plt.subplots(figsize=(12,6))
  ax.plot(time,alt, '-')
  ax.plot(time,[alt_media]*len(time), 'k--')
  ax.plot(time[i],alt[i], 'or')
  ax.set_title('Altitudine')
  ax.set_xlabel('Data')
  ax.set_ylabel('Altitudine (km)')
  ax.set_ylim(500,560)
  fig.autofmt_xdate()

  plt.show()

w = interactive(plot_altitude, i=intslider)
output = w.children[-1]
output.layout.height = '400px'
display(w)


L'altitudine media è 531.95 km


interactive(children=(IntSlider(value=0, continuous_update=False, description='Tempo (min)', max=50), Output(l…


---
# Puntamento del satellite

Dove guarda il satellite Fermi?
Il satellite guarda una porzione di cielo per volta (indicata dal cerchio rosso), ma si muove velocemente nel cielo ed in circa 3 ore ha gaurdato quasi completamente tutto il cielo.

La porzione di cielo che il satellite osserva si chiama "*Field of view*" ed nel caso del Fermi-LAT è circa il 20% dell'intero cielo!



In [ ]:
## Puntamento del satellite
# Mappa interattiva
# Esegui questo blocco e poi prova a modificare la variabile tempo usando la barra interattiva sul grafico
print("La traiettoria tratteggiata indica come è cambiata la direzione di puntamento nel tempo.")
print("Il puntino rosso indica la direzione al tempo selezionato e la curva rossa rappresenta il Field-of-view.")
print("Cambiare la variabile tempo per visualizzare il puntamento del satellite")
print("Alcuni valori da provare: 30, 50, 70, 100, 1000, 1500")

## Leggiamo le coordinate di puntamento
ra_z = tab_ft2['RA_SCZ']
dec_z = tab_ft2['DEC_SCZ']
c = SkyCoord(ra=ra_z*u.degree, dec=dec_z*u.degree, frame='fk5').galactic
lat_z = c.b
lon_z = c.l
max_entries=len(tab_ft2)-1
lat_s = 57.06
lon_s = 305.10
lat_s1 = -10.44
lon_s1 = 92.59


## Calcoliamo il campo di vista, detto Field-of-view (FoV)
FoV=2.4 #sr
theta = np.arccos(1-FoV/(2*np.pi))*180/np.pi
m = hp.read_map(allskymapHpx, verbose=False);

play = widgets.Play(value=0,min=0,max=max_entries,step=1,interval=500,description="Press play",disabled=False)
intslider = widgets.IntSlider(min=0, max=max_entries, step=1, value=0,description="Tempo (min)", continuous_update=False)
def plot_sky(t):
  fig = plt.figure(figsize=(8,6))
  hp.mollview(m, fig=fig.number, title='Il cielo gamma', norm=LogNorm(), badcolor='white');
  hp.graticule(verbose=False);
  hp.projplot(lon_z[:t], lat_z[:t], lonlat=True, color='k', ls='--');
  circle = SphericalCircle([lon_z[t],lat_z[t]],theta*u.deg, color='r')
  xy = circle.get_xy().T
  hp.projscatter(lon_z[t], lat_z[t], lonlat=True, color='r', marker='o');
  #hp.projscatter(lon_s, lat_s, lonlat=True, color='y', marker='*');
  hp.projscatter(lon_s1, lat_s1, lonlat=True, color='orange', marker='*');
  hp.projplot(xy[0],xy[1],lonlat=True,color='r');
  plt.show()

w = interactive(plot_sky, t=intslider)
output = w.children[-1]
output.layout.height = '380px'
display(w)


La traiettoria tratteggiata indica come è cambiata la direzione di puntamento nel tempo.
Il puntino rosso indica la direzione al tempo selezionato e la curva rossa rappresenta il Field-of-view.
Cambiare la variabile tempo per visualizzare il puntamento del satellite
Alcuni valori da provare: 30, 50, 70, 100, 1000, 1500


interactive(children=(IntSlider(value=0, continuous_update=False, description='Tempo (min)', max=10080), Outpu…

# THE END